# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import nltk, re, string
#nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import pickle


In [2]:
# load data from database
db = "DisasterResponse.db"
engine = create_engine('sqlite:///{}'.format(db))
df = pd.read_sql_table("Messages", engine)
#df = pd.read_sql("SELECT * FROM Messages;", engine)
X = df['message'].copy()
Y = df.iloc[:, 4:].copy()

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    #Normalize to remove punctuation and make all words lower case
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub(" ", text).lower()
    
    #Tokenize string.
    tokens = word_tokenize(text)
    
    #Remove stopwords
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    
    #Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(t).strip() for t in tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
print (X.shape)
print (Y.shape)

(26027,)
(26027, 36)


In [6]:
# perform train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

In [7]:
# either train new model or load existing model from pickle file
pipeline.fit(X_train, Y_train)
#pipeline = pickle.load(open("pipeline_fit.pkl", 'rb'))

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001C6A61660D0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [12]:
# save copy of model to save time in the future
pickle.dump(pipeline, open("pipeline_fit.pkl", 'wb'))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# Use pipeline to generate prediction and compare to Test data
Y_pred = pd.DataFrame(pipeline.predict(X_test), columns=Y.columns)

In [9]:
print(classification_report(Y_test, Y_pred, target_names=Y_test.columns, digits=2, zero_division=1))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4978
               request       0.85      0.50      0.63      1155
                 offer       1.00      0.00      0.00        27
           aid_related       0.78      0.68      0.73      2765
          medical_help       0.66      0.07      0.13       547
      medical_products       0.80      0.09      0.17       339
     search_and_rescue       0.64      0.03      0.06       205
              security       0.50      0.01      0.02       118
              military       0.67      0.06      0.10       213
           child_alone       1.00      1.00      1.00         0
                 water       0.92      0.35      0.50       447
                  food       0.84      0.62      0.71       729
               shelter       0.82      0.39      0.53       580
              clothing       0.87      0.12      0.22       105
                 money       0.60      

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001C6A61660D0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001C6A61660D0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {
    'clf__estimator__min_samples_split': [2, 3, 4]
    , 'clf__estimator__n_estimators': [10, 50, 100]
}
cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=2)

In [ ]:
cv.fit(X_train, Y_train)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total= 3.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.8min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total= 3.9min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total= 4.7min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total= 5.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 6.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 5.9min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, total= 3.1min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV]  

In [ ]:
print("\nBest Parameters:", cv.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
tuned_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=100, min_samples_split=3)))
])

In [ ]:
tuned_pipeline.fit(X_train, Y_train)

In [ ]:
Y_pred_tuned = pd.DataFrame(tuned_cv.predict(X_test), columns=Y.columns)

In [ ]:
print(classification_report(Y_test, Y_pred_tuned, target_names=Y_test.columns, digits=2, zero_division=1))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#Build new ML pipeline using KNeighborsClassifier with default settings
kn_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=KNeighborsClassifier()))
])

In [ ]:
#Fit the new ML pipeline
kn_pipeline.fit(X_train, Y_train)

In [ ]:
#Print classification report
Y_pred_kn = pd.DataFrame(kn_pipeline.predict(X_test), columns=Y.columns)
print(classification_report(Y_test, Y_pred_kn, target_names=Y_test.columns, digits=2, zero_division=1))

## Results: 
Overall, the KNeighborsClassifier performed worse than the RandomForestClassifier using default settings.  Given more time to perform GridSearch it's possible that the performance could be improved.

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(tuned_cv, open("classifier.pkl", 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.